## Capstone Project - UK Cities Restaurants Analysis

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


import data from "https://simplemaps.com/data/gb-cities" site which includes data about UK cities iterms of population and coordinates

In [4]:
uk_data = pd.read_csv("gb.csv")
uk_data.head()


,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,London,51.5072,-0.1275,United Kingdom,GB,"London, City of",primary,10979000,8825001
1,Birmingham,52.4800,-1.9025,United Kingdom,GB,Birmingham,NaN,2897303,1137100
2,Manchester,53.4794,-2.2453,United Kingdom,GB,Manchester,NaN,2705000,547627
3,Leeds,53.7997,-1.5492,United Kingdom,GB,Leeds,NaN,1901934,789194
4,Newcastle,55.0077,-1.6578,United Kingdom,GB,Newcastle upon Tyne,NaN,1650000,879996


In [5]:
uk_data.shape

(443, 9)

In [6]:
#get the coordinates of Londaon city
address = 'LONDON, UK'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


Create a map of all UK showing all projected 443 cities

In [7]:
# create map of UK using latitude and longitude values
map_uk = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city in zip(uk_data['lat'], uk_data['lng'], uk_data['city']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_uk)  
    
map_uk

In [8]:
#access creditionals of Foursquare API
CLIENT_ID = '05FPHY0IDWMACDHOL2M3OCIH2GESB3XX0LHWUOIWYAJCCSJ1' # your Foursquare ID
CLIENT_SECRET = 'VKVQKESB3Q5QLZOXYOXAQ32NTSGW5IOOUB3CCPSF51PLKZTK' # your Foursquare Secret
ACCESS_TOKEN = 'EZ0TGACZZ4ZEUMKAT3PGMQDDIZI13504MMVJ12BRDVSOPHHU'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [19]:
# a function to calculate get the venus per city
def getNearbyVenues(names, latitudes, longitudes,populations, radius=1000):
    search_query = 'Restaurant'
    venues_list=[]
    for name, lat, lng, population in zip(names, latitudes, longitudes, populations):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, lat, lng,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            population,
            v['name'], 
            v['location']['formattedAddress']
            ) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City Name', 
                  'Latitude', 
                  'Longitude',
                  'Population',
                  'Venue', 
                  'Venu Address']
    
    return(nearby_venues)

In [20]:
latitudes = uk_data['lat'] # city latitude value
longitudes = uk_data['lng'] # city longitude value
names = uk_data['city'] # city name
populations = uk_data['population']# city population
uk_venues = getNearbyVenues(names, latitudes, longitudes,populations)

uk_venues.head()

,City Name,Latitude,Longitude,Population,Venue,Venu Address
0,London,51.5072,-0.1275,10979000,Portrait Restaurant,"[St. Martin's Pl (Trafalgar Sq), London, Great..."
1,London,51.5072,-0.1275,10979000,The Elysee Restaurant and Roof Garden,"[13 Percy St, London, Greater London, W1T 1DP,..."
2,London,51.5072,-0.1275,10979000,London Chinatown Restaurant,"[27 Gerrard St, London, Greater London, W1D 6J..."
3,London,51.5072,-0.1275,10979000,New Loon Fung Chinese Restaurant,"[42-44 Gerrard St, London, Greater London, W1D..."
4,London,51.5072,-0.1275,10979000,River Restaurant,"[The Savoy, London, Greater London, WC2R 0EU, ..."


In [21]:
uk_venues.shape

(2561, 6)

calculate a statistics based on the retrieved data from Foursquare API.
calculate the number of restaurants per city.
calculate the value of restaurants per 100k person in each city based on the population

In [56]:
uk_venues['Restaurants Count']=uk_venues.groupby(by='City Name')['City Name'].transform('count')
uk_venues['Restaurants Per 100K Capita'] = uk_venues['Restaurants Count'] / (uk_venues['Population']/100000)
uk_venues.head()

,City Name,Latitude,Longitude,Population,Venue,Venu Address,Restaurants Count,Restaurants Per 100K Capita
0,London,51.5072,-0.1275,10979000,Portrait Restaurant,"[St. Martin's Pl (Trafalgar Sq), London, Great...",50,0.455415
1,London,51.5072,-0.1275,10979000,The Elysee Restaurant and Roof Garden,"[13 Percy St, London, Greater London, W1T 1DP,...",50,0.455415
2,London,51.5072,-0.1275,10979000,London Chinatown Restaurant,"[27 Gerrard St, London, Greater London, W1D 6J...",50,0.455415
3,London,51.5072,-0.1275,10979000,New Loon Fung Chinese Restaurant,"[42-44 Gerrard St, London, Greater London, W1D...",50,0.455415
4,London,51.5072,-0.1275,10979000,River Restaurant,"[The Savoy, London, Greater London, WC2R 0EU, ...",50,0.455415


In [37]:
uk_venues.shape

(2561, 8)

prepare the dataframe for clustering 

In [38]:
uk_stat_data = uk_venues.drop_duplicates(subset=['City Name'])
uk_stat_data.drop(['Latitude', 'Longitude', 'Venue', 'Venu Address'], axis=1, inplace = True)
uk_stat_data.head()

C:\Users\rashwan\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,City Name,Population,Restaurants Count,Restaurants Per 100K Capita
0,London,10979000,50,0.455415
50,Birmingham,2897303,50,1.725743
100,Manchester,2705000,50,1.848429
150,Leeds,1901934,34,1.787654
184,Newcastle,1650000,1,0.060606


In [42]:
uk_stat_data.reset_index(drop=True, inplace =True)
uk_stat_data.shape

(374, 4)

# Cluster Cities

Run k-means to cluster the neighborhood into 5 clusters.

In [43]:
# set number of clusters
kclusters = 5

uk_grouped_clustering = uk_stat_data.drop('City Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(uk_grouped_clustering)

In [44]:
kmeans.labels_ # see generated lables

array([1, 4, 4, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [45]:
# add clustering labels
uk_stat_data.insert(0, 'Cluster Labels', kmeans.labels_)

uk_merged = uk_venues.drop_duplicates(subset=['City Name'])
uk_merged.drop(['Population', 'Restaurants Count', 'Restaurants Per 100K Capita', 'Venue', 'Venu Address'], axis=1, inplace = True)

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
uk_merged = uk_merged.join(uk_stat_data.set_index('City Name'), on='City Name')

uk_merged.head()

C:\Users\rashwan\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,City Name,Latitude,Longitude,Cluster Labels,Population,Restaurants Count,Restaurants Per 100K Capita
0,London,51.5072,-0.1275,1,10979000,50,0.455415
50,Birmingham,52.4800,-1.9025,4,2897303,50,1.725743
100,Manchester,53.4794,-2.2453,4,2705000,50,1.848429
150,Leeds,53.7997,-1.5492,2,1901934,34,1.787654
184,Newcastle,55.0077,-1.6578,2,1650000,1,0.060606


In [47]:
uk_merged.reset_index(drop=True, inplace =True)
uk_merged.head()

,City Name,Latitude,Longitude,Cluster Labels,Population,Restaurants Count,Restaurants Per 100K Capita
0,London,51.5072,-0.1275,1,10979000,50,0.455415
1,Birmingham,52.4800,-1.9025,4,2897303,50,1.725743
2,Manchester,53.4794,-2.2453,4,2705000,50,1.848429
3,Leeds,53.7997,-1.5492,2,1901934,34,1.787654
4,Newcastle,55.0077,-1.6578,2,1650000,1,0.060606


creat a map of clusters excluding the 1st cluster

In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(uk_merged['Latitude'], uk_merged['Longitude'], uk_merged['City Name'], uk_merged['Cluster Labels']):
    if cluster == 0:
        continue
        
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1

In [50]:
uk_merged.loc[uk_merged['Cluster Labels'] == 0, uk_merged.columns[[0] + list(range(5, uk_merged.shape[1]))]]

,City Name,Restaurants Count,Restaurants Per 100K Capita
17,Stoke-on-Trent,4,1.073033
18,Coventry,24,6.617221
19,Reading,14,4.152972
20,Belfast,35,10.483091
21,Derby,15,5.545287
22,Plymouth,27,10.077710
23,Wolverhampton,12,4.716870
24,Abertawe,20,8.146640
25,Milton Keynes,6,2.609365
26,Aberdeen,15,6.989423


Cluster 2

In [51]:
uk_merged.loc[uk_merged['Cluster Labels'] == 1, uk_merged.columns[[0] + list(range(5, uk_merged.shape[1]))]]

,City Name,Restaurants Count,Restaurants Per 100K Capita
0,London,50,0.455415


Cluster 3

In [52]:
uk_merged.loc[uk_merged['Cluster Labels'] == 2, uk_merged.columns[[0] + list(range(5, uk_merged.shape[1]))]]

,City Name,Restaurants Count,Restaurants Per 100K Capita
3,Leeds,34,1.787654
4,Newcastle,1,0.060606
5,Birstall,2,0.140428


Cluster 4

In [53]:
uk_merged.loc[uk_merged['Cluster Labels'] == 3, uk_merged.columns[[0] + list(range(5, uk_merged.shape[1]))]]

,City Name,Restaurants Count,Restaurants Per 100K Capita
6,Glasgow,37,3.755240
7,Liverpool,50,5.786220
8,Portsmouth,10,1.168663
9,Southampton,27,3.155795
10,Nottingham,31,4.033108
11,Bristol,14,1.933702
12,Sheffield,26,3.793582
13,Kingston upon Hull,10,1.744287
14,Leicester,29,5.698386
15,Edinburgh,50,10.244852


Cluster 5

In [54]:
uk_merged.loc[uk_merged['Cluster Labels'] == 4, uk_merged.columns[[0] + list(range(5, uk_merged.shape[1]))]]

,City Name,Restaurants Count,Restaurants Per 100K Capita
1,Birmingham,50,1.725743
2,Manchester,50,1.848429


# After analysing the data, I found that there are clusters that we can see it is promising for opening a restaurant business in the related cities.
like clusters 2,3 and 5.